# Money Laundering Route Analysis Notebook

This notebook demonstrates how to use SolanaGuard to identify and analyze money laundering routes on the Solana blockchain.

## Overview

Money laundering on Solana typically follows several patterns:
1. **Layering**: Splitting and recombining funds through multiple wallets
2. **Cross-chain bridges**: Moving assets to different blockchains to break traceability
3. **Mixer services**: Using anonymization services to break the transaction trail
4. **CEX hopping**: Depositing and withdrawing from different exchanges

In [ ]:
# Import required libraries
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from datetime import datetime, timedelta

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import project config
import config

# Import SolanaGuard utilities with correct module path
from analysis.utils.address_utils import classify_address, detect_money_laundering_patterns
from analysis.utils.graph_utils import TransactionFlowGraph, build_money_laundering_graph, analyze_exfiltration_routes
from analysis.utils.risk_scoring import calculate_address_risk
from analysis.utils.visualization import visualize_money_laundering_routes

# Import collectors
from data_collection.collectors.helius_collector import HeliusCollector
from data_collection.collectors.rugcheck_collector import RugCheckCollector
from data_collection.collectors.vybe_collector import VybeCollector
from data_collection.collectors.range_collector import RangeCollector

# Configure plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Create output directories if they don't exist
os.makedirs(config.DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(config.DATA_DIR, "output"), exist_ok=True)
os.makedirs(os.path.join(config.DATA_DIR, "visualizations"), exist_ok=True)

ModuleNotFoundError: No module named 'config'

## Initialize API Collectors

First, we initialize the necessary API collectors to gather data from various sources:

In [ ]:
# Initialize collectors - using try/except to handle potential API errors
try:
    helius = HeliusCollector()
    range_api = RangeCollector()
    vybe = VybeCollector()
    
    print("Collectors initialized successfully")
except Exception as e:
    print(f"Error initializing collectors: {e}")

2025-04-25 00:29:55,726 - helius_collector - INFO - Initialized Helius collector
2025-04-25 00:29:55,728 - range_collector - INFO - Initialized Range collector
2025-04-25 00:29:55,730 - vybe_collector - INFO - Initialized Vybe collector
2025-04-25 00:29:55,728 - range_collector - INFO - Initialized Range collector
2025-04-25 00:29:55,730 - vybe_collector - INFO - Initialized Vybe collector


Collectors initialized successfully


## Analyze Known Money Laundering Addresses

Let's start by analyzing a few known high-risk addresses to establish baseline patterns:

In [ ]:
# Example addresses to analyze
# These are real addresses associated with money laundering/hacks on Solana
suspicious_addresses = [
    "EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ", # Wormhole Exploiter Address
    "DKZYkQzTq4UgQx8hK3YHyJjnhGckZbWUxWoFCzga7VdC", # Mango Markets Exploiter Address
    "FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV"  # Solend Exploiter Address
]

# Function to analyze a suspicious address
def analyze_suspicious_address(address):
    print(f"\nAnalyzing address: {address}")
    
    # Step 1: Get basic risk information
    try:
        risk_info = range_api.get_address_risk_score(address)
        print(f"Risk score: {risk_info.get('risk_score', 'N/A')}")
    except Exception as e:
        print(f"Error getting risk score: {e}")
        risk_info = {}
    
    # Step 2: Get transaction history
    try:
        tx_history = helius.fetch_transaction_history(address, limit=100)
        print(f"Fetched {len(tx_history)} transactions")
    except Exception as e:
        print(f"Error fetching transaction history: {e}")
        tx_history = []
    
    # Step 3: Get token transfers
    try:
        token_transfers = helius.analyze_token_transfers(address, limit=100)
        print(f"Fetched {len(token_transfers)} token transfers")
    except Exception as e:
        print(f"Error analyzing token transfers: {e}")
        token_transfers = []
    
    # Step 4: Analyze money laundering routes
    try:
        ml_routes = range_api.analyze_money_laundering_routes(address)
        print(f"Detected {len(ml_routes)} money laundering routes")
    except Exception as e:
        print(f"Error analyzing money laundering routes: {e}")
        ml_routes = []
    
    results = {
        "address": address,
        "risk_info": risk_info,
        "tx_history": tx_history,
        "token_transfers": token_transfers,
        "ml_routes": ml_routes
    }
    
    return results

# Analyze each address
analysis_results = {}
for address in suspicious_addresses:
    analysis_results[address] = analyze_suspicious_address(address)

2025-04-25 00:29:55,746 - range_collector - INFO - Getting risk score for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ on solana



Analyzing address: EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ


2025-04-25 00:29:59,014 - range_collector - ERROR - Failed to make API request: 401 Client Error: Unauthorized for url: https://api.range.org/v1/risk/address?address=EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ&network=solana
2025-04-25 00:29:59,016 - helius_collector - INFO - Fetching transaction history for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ (limit: 100)
2025-04-25 00:29:59,017 - helius_collector - INFO - Getting signatures for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ (limit: 100)
2025-04-25 00:29:59,016 - helius_collector - INFO - Fetching transaction history for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ (limit: 100)
2025-04-25 00:29:59,017 - helius_collector - INFO - Getting signatures for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ (limit: 100)
2025-04-25 00:29:59,093 - helius_collector - ERROR - Failed to make RPC request: 401 Client Error: Unauthorized for url: https://mainnet.helius-rpc.com/?api-key=None
2025-04-25 00:29:59,093 - helius_collector - ERROR - Fai

Error getting risk score: Request failed: 401 Client Error: Unauthorized for url: https://api.range.org/v1/risk/address?address=EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ&network=solana
Error fetching transaction history: Request failed: 401 Client Error: Unauthorized for url: https://mainnet.helius-rpc.com/?api-key=None


2025-04-25 00:29:59,260 - helius_collector - ERROR - Failed to make RPC request: 401 Client Error: Unauthorized for url: https://mainnet.helius-rpc.com/?api-key=None
2025-04-25 00:29:59,261 - range_collector - INFO - Analyzing money laundering routes for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ
2025-04-25 00:29:59,262 - range_collector - INFO - Getting risk score for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ on solana
2025-04-25 00:29:59,261 - range_collector - INFO - Analyzing money laundering routes for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ
2025-04-25 00:29:59,262 - range_collector - INFO - Getting risk score for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ on solana
2025-04-25 00:29:59,407 - range_collector - ERROR - Failed to make API request: 401 Client Error: Unauthorized for url: https://api.range.org/v1/risk/address?address=EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ&network=solana
2025-04-25 00:29:59,408 - range_collector - WARNING - Failed to get risk score for

...

Error getting risk score: Request failed: 401 Client Error: Unauthorized for url: https://api.range.org/v1/risk/address?address=FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV&network=solana
Error fetching transaction history: Request failed: 401 Client Error: Unauthorized for url: https://mainnet.helius-rpc.com/?api-key=None


2025-04-25 00:30:03,166 - helius_collector - ERROR - Failed to make RPC request: 401 Client Error: Unauthorized for url: https://mainnet.helius-rpc.com/?api-key=None
2025-04-25 00:30:03,167 - range_collector - INFO - Analyzing money laundering routes for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV
2025-04-25 00:30:03,167 - range_collector - INFO - Getting risk score for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV on solana
2025-04-25 00:30:03,167 - range_collector - INFO - Analyzing money laundering routes for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV
2025-04-25 00:30:03,167 - range_collector - INFO - Getting risk score for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV on solana


Error analyzing token transfers: Request failed: 401 Client Error: Unauthorized for url: https://mainnet.helius-rpc.com/?api-key=None


2025-04-25 00:30:03,417 - range_collector - ERROR - Failed to make API request: 401 Client Error: Unauthorized for url: https://api.range.org/v1/risk/address?address=FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV&network=solana
2025-04-25 00:30:03,418 - range_collector - WARNING - Failed to get risk score for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV: Request failed: 401 Client Error: Unauthorized for url: https://api.range.org/v1/risk/address?address=FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV&network=solana
2025-04-25 00:30:03,420 - range_collector - INFO - Getting transactions for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV on solana
2025-04-25 00:30:03,418 - range_collector - WARNING - Failed to get risk score for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV: Request failed: 401 Client Error: Unauthorized for url: https://api.range.org/v1/risk/address?address=FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV&network=solana
2025-04-25 00:30:03,420 - range_collector - INFO - Getting tr

Detected 0 money laundering routes


## Identify Money Laundering Patterns

Now that we have the data, let's identify specific money laundering patterns in the transaction history:

In [ ]:
# Identify money laundering patterns for each address
ml_patterns = {}

for address, data in analysis_results.items():
    ml_patterns[address] = detect_money_laundering_patterns(
        address,
        data["tx_history"],
        data["token_transfers"],
        data["risk_info"],
        data["ml_routes"]
    )
    
    # Display patterns detected
    print(f"\nPatterns detected for {address}:")
    for pattern in ml_patterns[address]["patterns"]:
        print(f"- {pattern['type']}: {pattern['description']} (confidence: {pattern['confidence']:.2f})")


Patterns detected for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ:

Patterns detected for DKZYkQzTq4UgQx8hK3YHyJjnhGckZbWUxWoFCzga7VdC:

Patterns detected for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV:


## Visualize Money Laundering Routes

Let's visualize the money laundering routes for better understanding:

In [ ]:
# Visualize money laundering routes for each address
for address, data in analysis_results.items():
    if data["ml_routes"]:
        # Build money laundering graph
        ml_graph = build_money_laundering_graph(data["ml_routes"])
        
        # Export graph to JSON for visualization
        graph_data = ml_graph.export_to_json()
        
        # Save graph data for later use
        with open(f"../../data/output/ml_graph_{address}.json", "w") as f:
            json.dump(graph_data, f)
        
        # Visualize money laundering routes
        viz_file = visualize_money_laundering_routes(
            data["ml_routes"],
            output_file=f"../../data/visualizations/ml_routes_{address}.png"
        )
        
        # Display visualization
        if viz_file:
            from IPython.display import Image
            display(Image(filename=viz_file))
        
        # Analyze potential exfiltration routes
        exfiltration_routes = analyze_exfiltration_routes(ml_graph, address)
        
        print(f"\nExfiltration routes for {address}:")
        for route in exfiltration_routes[:3]:  # Show top 3 routes
            print(f"- Route to {route['target_address']} (risk: {route['risk_score']})")
            print(f"  Path: {' -> '.join([route['source_address']] + route['intermediate_addresses'] + [route['target_address']])}")
            print(f"  Total volume: ${route['total_volume_usd']:.2f}")
    else:
        print(f"\nNo money laundering routes found for {address}")


No money laundering routes found for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ

No money laundering routes found for DKZYkQzTq4UgQx8hK3YHyJjnhGckZbWUxWoFCzga7VdC

No money laundering routes found for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV


## Calculate Risk Scores

Let's calculate comprehensive risk scores for the addresses:

In [ ]:
# Calculate risk scores for each address
risk_scores = {}

for address, data in analysis_results.items():
    risk_scores[address] = calculate_address_risk(
        address,
        data["tx_history"],
        data["token_transfers"],
        data["risk_info"],
        data["ml_routes"]
    )
    
    # Display risk score and factors
    print(f"\nRisk assessment for {address}:")
    print(f"- Overall score: {risk_scores[address]['risk_score']:.2f}")
    print(f"- Risk level: {risk_scores[address]['risk_level']}")
    print("- Top risk factors:")
    for factor in risk_scores[address]['risk_factors'][:3]:  # Show top 3 factors
        print(f"  * {factor['name']}: {factor['description']} (score: {factor['score']})")


Risk assessment for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ:
- Overall score: 0.00
- Risk level: Unknown
- Top risk factors:
  * Placeholder Risk: No real risk evaluation available (score: 0)

Risk assessment for DKZYkQzTq4UgQx8hK3YHyJjnhGckZbWUxWoFCzga7VdC:
- Overall score: 0.00
- Risk level: Unknown
- Top risk factors:
  * Placeholder Risk: No real risk evaluation available (score: 0)

Risk assessment for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV:
- Overall score: 0.00
- Risk level: Unknown
- Top risk factors:
  * Placeholder Risk: No real risk evaluation available (score: 0)


## Identify Related Addresses

Let's identify other addresses that might be part of the same money laundering network:

In [ ]:
# Analyze transaction flows to find related addresses
for address, data in analysis_results.items():
    if data["token_transfers"]:
        # Create transaction flow graph
        flow_graph = TransactionFlowGraph()
        flow_graph.add_token_transfers(data["token_transfers"])
        
        # Get graph centrality metrics
        centrality = flow_graph.calculate_centrality()
        
        # Sort by centrality metrics to find important addresses
        important_addresses = sorted(
            [(node, metrics["pagerank"]) for node, metrics in centrality.items()],
            key=lambda x: x[1],
            reverse=True
        )[:10]  # Top 10 by PageRank
        
        print(f"\nImportant addresses in the network of {address}:")
        for addr, score in important_addresses:
            if addr != address:  # Skip the original address
                print(f"- {addr} (centrality: {score:.4f})")
                
        # Find communities
        communities = flow_graph.identify_communities()
        
        print(f"\nDetected {len(communities)} communities in the network:")
        for comm_id, nodes in communities.items():
            if address in nodes:  # Focus on communities containing our target address
                print(f"- Community {comm_id}: {len(nodes)} addresses")
                # Show a few example addresses (excluding our target)
                example_nodes = [node for node in nodes[:5] if node != address]
                if example_nodes:
                    print(f"  Examples: {', '.join(example_nodes)}")
    else:
        print(f"\nInsufficient data to analyze the network for {address}")


Insufficient data to analyze the network for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ

Insufficient data to analyze the network for DKZYkQzTq4UgQx8hK3YHyJjnhGckZbWUxWoFCzga7VdC

Insufficient data to analyze the network for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV


## Conclusions and Report Generation

Let's summarize our findings and generate a report:

In [ ]:
# Generate summary report
report = ["# Money Laundering Analysis Report\n"]
report.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

report.append("## Addresses Analyzed\n")
for address in suspicious_addresses:
    risk_level = risk_scores[address]['risk_level'] if address in risk_scores else "Unknown"
    report.append(f"- **{address}**: Risk Level: {risk_level}")

for address in suspicious_addresses:
    report.append(f"\n## Analysis for {address}\n")
    
    # Risk assessment
    if address in risk_scores:
        report.append(f"### Risk Assessment\n")
        report.append(f"- Overall risk score: {risk_scores[address]['risk_score']:.2f}")
        report.append(f"- Risk level: {risk_scores[address]['risk_level']}\n")
        
        report.append("#### Risk Factors\n")
        for factor in risk_scores[address]['risk_factors']:
            report.append(f"- **{factor['name']}**: {factor['description']} (score: {factor['score']})")
    
    # Money laundering patterns
    if address in ml_patterns:
        report.append(f"\n### Money Laundering Patterns\n")
        
        if ml_patterns[address]["is_suspicious"]:
            report.append("This address shows suspicious transaction patterns:\n")
            
            for pattern in ml_patterns[address]["patterns"]:
                report.append(f"- **{pattern['type']}**: {pattern['description']} (confidence: {pattern['confidence']:.2f})")
        else:
            report.append("No suspicious patterns detected for this address.")
    
    # Money laundering routes
    if address in analysis_results and analysis_results[address]["ml_routes"]:
        report.append(f"\n### Money Laundering Routes\n")
        
        ml_routes = analysis_results[address]["ml_routes"]
        report.append(f"Detected {len(ml_routes)} potential money laundering routes:\n")
        
        for i, route in enumerate(ml_routes[:5], 1):
            report.append(f"#### Route {i}\n")
            report.append(f"- **From**: {route.get('source_address', 'Unknown')}")
            report.append(f"- **To**: {route.get('target_address', 'Unknown')}")
            report.append(f"- **Type**: {route.get('flow_type', 'Unknown')}")
            report.append(f"- **Risk Score**: {route.get('risk_score', 'Unknown')}")
            report.append(f"- **Amount**: ${route.get('amount_usd', 0):.2f}\n")
            
        report.append(f"![Money Laundering Routes](../../data/visualizations/ml_routes_{address}.png)")
    else:
        report.append("\n### Money Laundering Routes\n")
        report.append("No money laundering routes detected for this address.")

# Save report to file
report_path = f"../../reports/money_laundering_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
os.makedirs("../../reports", exist_ok=True)

with open(report_path, "w") as f:
    f.write("\n".join(report))

print(f"\nReport generated and saved to {report_path}")

# Display report in notebook
from IPython.display import Markdown
display(Markdown("\n".join(report)))


Report generated and saved to ../../reports/money_laundering_report_20250425_003004.md


# Money Laundering Analysis Report

Generated on: 2025-04-25 00:30:04

## Addresses Analyzed

- **EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ**: Risk Level: Unknown
- **DKZYkQzTq4UgQx8hK3YHyJjnhGckZbWUxWoFCzga7VdC**: Risk Level: Unknown
- **FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV**: Risk Level: Unknown

## Analysis for EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ

### Risk Assessment

- Overall risk score: 0.00
- Risk level: Unknown

#### Risk Factors

- **Placeholder Risk**: No real risk evaluation available (score: 0)

### Money Laundering Patterns

No suspicious patterns detected for this address.

### Money Laundering Routes

No money laundering routes detected for this address.

## Analysis for DKZYkQzTq4UgQx8hK3YHyJjnhGckZbWUxWoFCzga7VdC

### Risk Assessment

- Overall risk score: 0.00
- Risk level: Unknown

#### Risk Factors

- **Placeholder Risk**: No real risk evaluation available (score: 0)

### Money Laundering Patterns

No suspicious patterns detected for this address.

### Money Laundering Routes

No money laundering routes detected for this address.

## Analysis for FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV

### Risk Assessment

- Overall risk score: 0.00
- Risk level: Unknown

#### Risk Factors

- **Placeholder Risk**: No real risk evaluation available (score: 0)

### Money Laundering Patterns

No suspicious patterns detected for this address.

### Money Laundering Routes

No money laundering routes detected for this address.